Hay que hacer bonito esto pero es pa poner la direccion de la api 


In [24]:
import requests as rq
import pandas as pd

In [25]:
lista_paises = [["JPN", "1998:2006", "WC"], ["KOR", "1998:2006", "WC"], 
["DEU", "2002:2010", "WC"], ["ZAF", "2006:2014", "WC"], ["BRA", "2010:2018", "WC"], ["RUS", "2014:2020", "WC"], 
["GRC", "2000:2008", "OL"], ["CHN", "2004:2012", "OL"], ["GBR", "2008:2016", "OL"], ["BRA", "2012:2020", "OL"]]

In [26]:
lista_mundial = []
lista_ol = []

for pais in lista_paises:
    iso_pais = []
    ano = []
    pib = []
    personas = []
    gasto = []
    ingreso = []

    fecha = pais[1]
    iso_code = pais[0]

    url_pib = f"http://api.worldbank.org/v2/country/{iso_code}/indicator/NY.GDP.PCAP.PP.CD?format=json&frequency=Y&date={fecha}"
    r = rq.get(url_pib)
    json_pib = r.json()

    url_personas = f"http://api.worldbank.org/v2/country/{iso_code}/indicator/ST.INT.ARVL?format=json&frequency=Y&date={fecha}"
    r = rq.get(url_personas)
    json_personas = r.json()

    url_gasto = f"http://api.worldbank.org/v2/country/{iso_code}/indicator/ST.INT.XPND.CD?format=json&frequency=Y&date={fecha}"
    r = rq.get(url_gasto)
    json_gasto = r.json()  

    url_ingreso = f"http://api.worldbank.org/v2/country/{iso_code}/indicator/ST.INT.RCPT.CD?format=json&frequency=Y&date={fecha}"
    r = rq.get(url_ingreso)
    json_ingreso = r.json()  

    json_pib.pop(0)
    for i in json_pib[0]:
        iso_pais.append(i['countryiso3code'])
        ano.append(i['date'])
        pib.append(i['value'])

    json_personas.pop(0)
    for i in json_personas[0]:
        personas.append(i["value"])

    json_gasto.pop(0)
    for i in json_gasto[0]:
        gasto.append(i["value"])

    json_ingreso.pop(0)
    for i in json_ingreso[0]:
        ingreso.append(i["value"])        
    
    df = pd.DataFrame(columns=['pais', 'fecha', 'PIB_per_capita', 'personas_llegadas_al_pais', 'gasto_turismo', 'ingreso_turismo'])
    df["pais"] = iso_pais
    df["fecha"] = ano
    df["PIB_per_capita"] = pib
    df['personas_llegadas_al_pais'] = personas
    df['gasto_turismo'] = gasto
    df['ingreso_turismo'] = ingreso

    if pais[2] == "WC":
        lista_mundial.append(df)

    else:
        lista_ol.append(df)
        


In [28]:
lista_mundial[5]

,pais,fecha,PIB_per_capita,personas_llegadas_al_pais,gasto_turismo,ingreso_turismo
0,RUS,2020,29916.013672,6359000,10800000000,4961000000
1,RUS,2019,30067.740234,24419000,40611000000,17235000000
2,RUS,2018,28821.261719,24551000,38791000000,18735000000
3,RUS,2017,25926.443359,24390000,35584000000,14938000000
4,RUS,2016,24128.085938,24571000,27654000000,12822000000
5,RUS,2015,24085.324219,33729000,38432000000,13186000000
6,RUS,2014,25761.648438,32421000,55383000000,19451000000
